### Power plant cooling

Cooling water leaves the condenser of a power plant and enters a wet cooling tower at 35°C at a rate of 100 kg/s. Water is cooled to 22°C in the cooling tower by air that enters the tower at 1 atm, 20°C, and 60 percent relative humidity and leaves saturated at 30°C. 

Neglecting the power input to the fan, determine: 
- (a) the volume flow rate of air into the cooling tower and
- (b) the mass flow rate of the required makeup water.

In [3]:
Tw3_C = 35
mdot_w_kgPerS = 100

Tw4_C = 22

pAir_Pa = 101325
Tair1_C = 20
RH1 = 60

Tair2_C = 30
RH2 = 100

Defining states:
- 1: undersaturated air entering the cooling tower
- 2: saturated air exiting the cooling tower
- 3: warm water entering the cooling tower
- 4: cool water exiting the cooling tower

In [4]:
try:
    from iapws import IAPWS97
except ImportError:
    !pip install iapws
    from iapws import IAPWS97

In [6]:
# state 3
state3 = IAPWS97(T=Tw3_C+273.15, x=0)
h3_kJperKg = state3.h; print(h3_kJperKg)

146.6448016353406


In [7]:
# state 4
state4 = IAPWS97(T=Tw4_C+273.15, x=0)
h4_kJperKg = state4.h; print(h4_kJperKg)

92.28904176486171


In [8]:
try:
    from pyfluids import HumidAir, InputHumidAir
except ImportError:
    !pip install pyfluids
    from pyfluids import HumidAir, InputHumidAir

In [11]:
state1 = HumidAir().with_state(
    InputHumidAir.pressure(pAir_Pa),
    InputHumidAir.temperature(Tair1_C),
    InputHumidAir.relative_humidity(RH1),
    #InputHumidAir.wet_bulb_temperature(Twb_C),
)

h1_JperKg = state1.enthalpy
h1_kJperKg = h1_JperKg/1e3; print(h1_kJperKg)
w1 = state1.humidity; print(w1)
v1_m3PerKg = state1.specific_volume; print(v1_m3PerKg)

42.00662487579089
0.008773013888634404
0.8345067876192733


In [14]:
state2 = HumidAir().with_state(
    InputHumidAir.pressure(pAir_Pa),
    InputHumidAir.temperature(Tair2_C),
    InputHumidAir.relative_humidity(RH2),
    #InputHumidAir.wet_bulb_temperature(Twb_C),
)

h2_JperKg = state2.enthalpy; print(h2_JperKg)
h2_kJperKg = h2_JperKg/1e3
w2 = state2.humidity; print(w2)

97349.63193164689
0.027332863482547005


Mass balance:
$$\dot{m}_\text{a1} = \dot{m}_\text{a2} = \dot{m}_\text{a} $$

$$\dot{m}_3 + \dot{m}_\text{a1}\omega_1 = \dot{m}_4 + \dot{m}_\text{a2}\omega_2$$

Energy balance:

$$\dot{m}_\text{a1}h_1 + \dot{m}_\text{3}h_3 = \dot{m}_\text{a2}h_2 + \dot{m}_\text{4}h_4 $$

$$\dot{m}_\text{a}(h_1 - h_2) = \dot{m}_\text{4}h_4 - \dot{m}_\text{3}h_3 $$

The losses due to evaporation that must be "made up" are: 

$$\dot{m}_\text{makeup} = \dot{m}_3 - \dot{m}_4 = \dot{m}_\text{a} (\omega_2 - \omega_1)  $$


Re-writing our energy balance yields:

$$\dot{m}_\text{a}(h_1 - h_2) = (\dot{m}_\text{3} - \dot{m}_\text{makeup})h_4 - \dot{m}_\text{3}h_3 $$

Substituting the definition for the makeup mass flow rate:






$$\dot{m}_\text{a}(h_1 - h_2) = h_4\dot{m}_\text{3} - h_4(\dot{m}_\text{a} (\omega_2 - \omega_1) ) - \dot{m}_\text{3}h_3 $$

$$\dot{m}_\text{a}[(h_1 - h_2) + h_4(\omega_2 - \omega_1)] = h_4\dot{m}_\text{3} - \dot{m}_\text{3}h_3 $$

$$\dot{m}_\text{a} = \frac{h_4\dot{m}_\text{3} - \dot{m}_\text{3}h_3}{[(h_1 - h_2) + h_4(\omega_2 - \omega_1)]} $$

Rearranging:
$$\dot{m}_\text{a} = \frac{\dot{m}_\text{3}(h_3 - h_4)}{[(h_2 - h_1) - h_4(\omega_2 - \omega_1)]} $$

In [19]:
mdotAir_kgPerS = (mdot_w_kgPerS*(h3_kJperKg - h4_kJperKg))/((h2_kJperKg - h1_kJperKg)-(h4_kJperKg*(w2-w1))); print(mdotAir_kgPerS)

101.35301455247969


In [20]:
Vdot_m3PerS = mdotAir_kgPerS*v1_m3PerKg

print('(a) Volume flow rate of air into the cooling tower: {0:1.2f} m3/s'.format(Vdot_m3PerS))

(a) Volume flow rate of air into the cooling tower: 84.58 m3/s


In [21]:
mdotMakeup_kgPerS = mdotAir_kgPerS*(w2 - w1)

print('(b) mdotMakeup_kgPerS: {0:1.2f} kg/s'.format(mdotMakeup_kgPerS))

(b) mdotMakeup_kgPerS: 1.88 kg/s
